In [297]:
from bs4 import BeautifulSoup
import re
import requests

<h1> Scraping our data</h1>
<h3> First thing we need to do to start the project would be to gather our data</h3>
<p> We would like to find a reliable website of a very known fighting organization , this way we can trust the data since it is an official fighting organization </p>
<p>We are going to scrape ESPN official website for getting our data </p>
<a href="http://www.espn.com/mma/fighters">  Link to ESPN's website where we can observe the fighter list </a>

In [298]:
user_agent = {'User-agent': 'Mozilla/5.0'}

<h3> First we would like to have a list with links for all pages of of the fither list </h3>
<h3> first page will contain fithers with name started with letter A</h3>
<h3> first page will contain fithers with name started with letter Z</h3>

In [299]:
url1="http://www.espn.com/mma/fighters"
response1 = requests.get(url1,headers=user_agent)
pagesAToZ=[]
# we would like to add the first page (the page that contains fighters with letter A)
pagesAToZ.append(url1)
soup1 = BeautifulSoup(response1.content, "html.parser")
div_t=soup1.find("div",attrs={"class":"mod-content"})
hred_t=div_t.findAll("a")
for t in hred_t :
    pagesAToZ.append(url1+t['href'])


<h3>Let's print the urls so we can make sure everything looks fine </h3>

In [300]:
print(pagesAToZ)

['http://www.espn.com/mma/fighters', 'http://www.espn.com/mma/fighters?search=b', 'http://www.espn.com/mma/fighters?search=c', 'http://www.espn.com/mma/fighters?search=d', 'http://www.espn.com/mma/fighters?search=e', 'http://www.espn.com/mma/fighters?search=f', 'http://www.espn.com/mma/fighters?search=g', 'http://www.espn.com/mma/fighters?search=h', 'http://www.espn.com/mma/fighters?search=i', 'http://www.espn.com/mma/fighters?search=j', 'http://www.espn.com/mma/fighters?search=k', 'http://www.espn.com/mma/fighters?search=l', 'http://www.espn.com/mma/fighters?search=m', 'http://www.espn.com/mma/fighters?search=n', 'http://www.espn.com/mma/fighters?search=o', 'http://www.espn.com/mma/fighters?search=p', 'http://www.espn.com/mma/fighters?search=q', 'http://www.espn.com/mma/fighters?search=r', 'http://www.espn.com/mma/fighters?search=s', 'http://www.espn.com/mma/fighters?search=t', 'http://www.espn.com/mma/fighters?search=u', 'http://www.espn.com/mma/fighters?search=v', 'http://www.espn.c

<h3> Another function we would like to writw is a functions that scrapes the most inner page that contain previews fights stats , we would like to get a path as an argument</h3>

In [301]:
def scrapeinnerPageForFightStats(path):
    
    wins =""
    loses =""
    decision = ""
    winByKo = ""
    loseByKo= ""
    winbySubmission=""
    loseBySubmission=""
    page_link='http://www.espn.com'+path
    print("pagelink is" , page_link )
    response= requests.get(page_link)
    soup = BeautifulSoup(response.content)
    regex_winsLosesDecision = "W-L-D(\d)-(\d)-(\d)"
    regex_ko="\(T\)KO(\d)-(\d)"
    regex_sub="SUB(\d)-(\d)"
    ul_t = soup.find("ul", attrs={"class":"StatBlock__Content flex list ph4 pv3 justify-between"}) 
    if(ul_t==None):
       print("No unordered list atrr")
    li_list=ul_t.findAll("li")
    for li in li_list:
       text=li.text
    #    print(text)
       winLoseDecStats_regex_res=re.search(regex_winsLosesDecision , text)
       koStats_regex_res=re.search(regex_ko , text)
       subStats_regex_res=re.search(regex_sub , text)
       if(winLoseDecStats_regex_res):
           wins=winLoseDecStats_regex_res.group(1)
           loses=winLoseDecStats_regex_res.group(2)
           decision=winLoseDecStats_regex_res.group(3)
        #    print(winLoseDecStats_regex_res)
       elif(koStats_regex_res):
           winByKo=koStats_regex_res.group(1)
           loseByKo=koStats_regex_res.group(2)
        #    print(koStats_regex_res)
       elif(subStats_regex_res):
           winbySubmission=subStats_regex_res.group(1)
           loseBySubmission=subStats_regex_res.group(2)
        #    print(subStats_regex_res)

    print(wins , loses , decision , winByKo , loseByKo, winbySubmission , loseBySubmission)
    return wins , loses , decision , winByKo , loseByKo, winbySubmission , loseBySubmission


<h3> Let's make sure we get the right return params and that the regexes are fine</h3>
<p> we will expect three matches and output params ('0', '1', '0', '0', '0', '0', '1') </p>

In [302]:
scrapeinnerPageForFightStats('/mma/fighter/_/id/3043549/niina-aaltonen')

pagelink is http://www.espn.com/mma/fighter/_/id/3043549/niina-aaltonen
0 1 0 0 0 0 1


('0', '1', '0', '0', '0', '0', '1')

<h3> Cool , We have a list of links to each page we want to scrape and the stats from the inner pages! </h3>
<h4>Now we would like to build a function for scraping each page  </h4>

<p>
/The structure of the page containes a long table with two columns . <br>
First Column - Name <br>
Sec Column - Country <br>

the Name is clickable and redirect us to another page with more details of the fighter
 </p>

In [303]:
def getTableRows(page_link):
    response= requests.get(page_link)
    soup = BeautifulSoup(response.content)
    table = soup.find(lambda tag: tag.name=='table') 
    rows = table.findAll(lambda tag: tag.name=='tr')
    # we would like to remove the first two rows bc it doesnt contain any data
    del rows[0]
    del rows[0]
    return rows

In [304]:
def runOnSingleDataRowGetNameCountryBio(row):
    #currently runs only on the first row
        full_name , link_to_bio , country = "" , "" ,""
        cells = row.findAll(lambda tag: tag.name=='td')
        if(cells == None or (not cells)):
            return 
        full_name = cells[0].find("a").text
        full_name=full_name.split(", ")
        first_name=full_name[0]
        last_name=full_name[1]
        link_to_bio = cells[0].find("a")["href"]
        country=cells[1].text
        full_name=first_name+" "+last_name 
        return full_name , link_to_bio ,country



In [305]:
def scrapeTheWholePage(page_url):
    rows=getTableRows(page_url)
    for row in rows:
        full_name  , bio_link , country = runOnSingleDataRowGetNameCountryBio(row)
        wins , loses , decision , winByKo , loseByKo, winbySubmission , loseBySubmission =scrapeinnerPageForFightStats(bio_link)
        print(full_name , bio_link , country)
        print(wins , loses , decision , winByKo , loseByKo, winbySubmission , loseBySubmission)
        # now we need to return DF with the whole page data
        #next were gonna gather all the dfs together and save it to a file
    
    

    

In [306]:
scrapeTheWholePage('http://www.espn.com/mma/fighters')

pagelink is http://www.espn.com/mma/fighter/_/id/3043549/niina-aaltonen
0 1 0 0 0 0 1
Aaltonen Niina /mma/fighter/_/id/3043549/niina-aaltonen 
0 1 0 0 0 0 1
pagelink is http://www.espn.com/mma/fighter/_/id/4898183/soufyan-aarab
0 1 0 0 0 0 0
Aarab Soufyan /mma/fighter/_/id/4898183/soufyan-aarab 
0 1 0 0 0 0 0
pagelink is http://www.espn.com/mma/fighter/_/id/2504991/tom-aaron
1 3 0 0 1 1 0
Aaron Tom /mma/fighter/_/id/2504991/tom-aaron USA
1 3 0 0 1 1 0
pagelink is http://www.espn.com/mma/fighter/_/id/4239497/tommy-aaron
9 5 0 3 3 2 0
Aaron Tommy /mma/fighter/_/id/4239497/tommy-aaron ESP
9 5 0 3 3 2 0
pagelink is http://www.espn.com/mma/fighter/_/id/3088828/joshua-aarons
0 2 0 0 0 0 2
Aarons Joshua /mma/fighter/_/id/3088828/joshua-aarons ESP
0 2 0 0 0 0 2
pagelink is http://www.espn.com/mma/fighter/_/id/3089919/mike-aarts
0 1 0 0 0 0 1
Aarts Mike /mma/fighter/_/id/3089919/mike-aarts ESP
0 1 0 0 0 0 1
pagelink is http://www.espn.com/mma/fighter/_/id/2511451/zyad-abada
0 2 0 0 0 0 2
Abada 

KeyboardInterrupt: 